In [38]:
q= '''
SELECT *
FROM
    (SELECT *,
    row_number()
    OVER
        (PARTITION BY "open_time"
        ORDER BY "close_time" DESC, Api_Invocation_Time DESC, write_time DESC)
    AS row_number
    FROM "sagemaker_featurestore"."btcpricedata_1704246894")
WHERE row_number = 1 ORDER BY close_time ASC;
'''

In [39]:
from sagemaker.feature_store.feature_group import FeatureGroup
import boto3
import pandas as pd
import numpy as np
import io
from sagemaker.session import Session
from sagemaker import get_execution_role
sagemaker_session = Session()

feature_group = FeatureGroup(name = 'BtcPriceData')

query = feature_group.athena_query()

table = query.table_name
table
default_s3_bucket_name = 'binance-btc-prices'

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [45]:

# run Athena query. The output is loaded to a Pandas dataframe.
dataset = pd.DataFrame()
query.run(query_string=q, output_location='s3://'+default_s3_bucket_name+'/query_results/')
query.wait()
dataset = query.as_dataframe()

In [46]:
dataset['open_time'] = pd.to_datetime(dataset['open_time'], unit='ms').dt.strftime('%Y-%m-%d %T')
dataset

,open_time,high_price,low_price,close_price,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,open_price,write_time,api_invocation_time,is_deleted,row_number
0,2019-09-09 00:00:00,10392.41,10376.45,10390.08,27.884150,1.567987e+09,2.896169e+05,626.0,12.492958,10381.24,2024-01-08 18:11:09.934,2024-01-08 18:05:58.000,False,1
1,2019-09-09 00:05:00,10393.94,10368.87,10372.31,36.750216,1.567988e+09,3.814708e+05,614.0,22.195335,10390.74,2024-01-08 18:11:07.429,2024-01-08 18:05:58.000,False,1
2,2019-09-09 00:10:00,10386.24,10368.88,10374.20,38.888045,1.567988e+09,4.035607e+05,640.0,19.145836,10372.31,2024-01-08 18:11:30.173,2024-01-08 18:05:58.000,False,1
3,2019-09-09 00:15:00,10387.83,10368.89,10384.89,24.997683,1.567988e+09,2.593937e+05,712.0,11.799247,10374.18,2024-01-08 18:11:10.880,2024-01-08 18:05:58.000,False,1
4,2019-09-09 00:20:00,10396.00,10382.40,10394.20,18.551736,1.567989e+09,1.927592e+05,571.0,9.729698,10386.25,2024-01-08 18:11:06.932,2024-01-08 18:05:58.000,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455099,2024-01-09 00:05:00,47080.23,47009.15,47031.18,154.558620,1.704759e+09,7.271087e+06,5979.0,91.207280,47039.99,2024-01-09 00:31:13.144,2024-01-09 00:26:16.000,False,1
455100,2024-01-09 00:10:00,47100.00,47012.39,47099.99,156.090540,1.704759e+09,7.345640e+06,6167.0,81.751510,47031.17,2024-01-09 00:31:08.899,2024-01-09 00:26:16.000,False,1
455101,2024-01-09 00:15:00,47150.00,47028.68,47060.00,322.398390,1.704760e+09,1.518189e+07,9130.0,153.485620,47099.99,2024-01-09 00:31:11.863,2024-01-09 00:26:15.000,False,1
455102,2024-01-09 00:20:00,47074.79,46923.32,46944.00,345.134360,1.704760e+09,1.621257e+07,7590.0,94.619050,47060.00,2024-01-09 00:31:13.144,2024-01-09 00:26:16.000,False,1


In [42]:
#dataset.to_csv('btc_prices.csv')

In [52]:
# univariate data preparation
from numpy import array
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [54]:
X,y  = split_sequence(sequence = dataset.close_price, n_steps = 10)

In [55]:
X 

array([[10390.08, 10372.31, 10374.2 , ..., 10391.29, 10388.87, 10375.54],
       [10372.31, 10374.2 , 10384.89, ..., 10388.87, 10375.54, 10387.7 ],
       [10374.2 , 10384.89, 10394.2 , ..., 10375.54, 10387.7 , 10388.03],
       ...,
       [43756.7 , 43732.46, 43759.7 , ..., 47040.  , 47031.18, 47099.99],
       [43732.46, 43759.7 , 43619.99, ..., 47031.18, 47099.99, 47060.  ],
       [43759.7 , 43619.99, 43773.54, ..., 47099.99, 47060.  , 46944.  ]])

In [60]:
# define model
from numpy import array
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_steps)) 
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=10, verbose=0)

ModuleNotFoundError: No module named 'keras'

In [1]:
import pandas as pd
pd.to_datetime(1.5679872E12, unit = 'ms')

Timestamp('2019-09-09 00:00:00')